In [21]:
import chat_pb2 as chat
import chat_pb2_grpc as rpc
import grpc
import threading
from tkinter import *
from tkinter import simpledialog

In [32]:
class Client:

    def __init__(self, u: str, window):
        # the frame to put ui components on
        self.window = window
        self.username = u
        # create a gRPC channel + stub
        channel = grpc.insecure_channel('localhost:9999')
        print(channel)
        self.conn = rpc.ChatServiceStub(channel)
        # create new listening thread for when new message streams come in
        threading.Thread(target=self.__listen_for_messages).start()
        self.__setup_ui()
        self.window.mainloop()

    def __listen_for_messages(self):
        """
        This method will be ran in a separate thread as the main/ui thread, because the for-in call is blocking
        when waiting for new messages
        """
        for note in self.conn.SendChatMessage(chat.ChatMessageFromServer()):  # this line will wait for new messages from the server!
            print(1)
            print("R[{}] {}".format(note.sender_username, note.message))  # debugging statement
            self.chat_list.insert(END, "[{}] {}\n".format(note.sender_username, note.message))  # add the message to the UI
            print(11)
            
    def send_message(self, event):
        """
        This method is called when user enters something into the textbox
        """
        message = self.entry_message.get()  # retrieve message from the UI
        if message !="":
            n = chat.ChatMessage(sender_username = self.username,message = message)  # create protobug message (called Note)
            self.conn.SendChatMessage(n)  # send the Note to the server
            print("S[{}] {}".format(n.sender_username, n.message))  # debugging statement

    def __setup_ui(self):
        self.chat_list = Text()
        self.chat_list.pack(side=TOP)
        self.lbl_username = Label(self.window, text=self.username)
        self.lbl_username.pack(side=LEFT)
        self.entry_message = Entry(self.window, bd=5)
        self.entry_message.bind('<Return>', self.send_message)
        self.entry_message.focus()
        self.entry_message.pack(side=BOTTOM)


if __name__ == '__main__':
    root = Tk()  # I just used a very simple Tk window for the chat UI, this can be replaced by anything
    frame = Frame(root, width=300, height=300)
    frame.pack()
    root.withdraw()
    username = None
    while username is None:
        # retrieve a username so we can distinguish all the different clients
        username = simpledialog.askstring("Username", "What's your username?", parent=root)
    root.deiconify()  # don't remember why this was needed anymore...
    c = Client(username, frame)  # this starts a client and thus a thread which keeps connection to server open

Exception in thread Thread-126:
Traceback (most recent call last):
  File "C:\Users\SALAHEDDINE\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\SALAHEDDINE\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\SALAHE~1\AppData\Local\Temp/ipykernel_8052/771509164.py", line 21, in __listen_for_messages
  File "C:\Users\SALAHEDDINE\AppData\Local\Programs\Python\Python39\lib\site-packages\grpc\_channel.py", line 426, in __next__
    return self._next()
  File "C:\Users\SALAHEDDINE\AppData\Local\Programs\Python\Python39\lib\site-packages\grpc\_channel.py", line 809, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception iterating requests!"
	debug_error_string = "None"
>


2
S[amine] hi
22
